### **Extraindo interações**

**Objetivo**: unir os valores relacionados a data em que os tweets foram postados para criar um relatório que seja possível visualizar o desempenho da palavra "data science" nos Tweets, como, por exemplo, a quantidade de comentários sobre o tema ao longo da semana.

In [18]:
from pyspark.sql import SparkSession

In [19]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

In [20]:
import os

parent_folder = "../../datalake/Silver/twitter_datascience/tweet"
subfolders = [os.path.join(parent_folder, folder) for folder in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, folder))]

df_tweet = spark.read.json(subfolders)

In [21]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [15]:
df_tweet.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       84|             92|2024-07-03T08:58:...| 61|        27|         85|         95|          100|Tweet fictício ge...|
|       61|             20|2024-07-03T17:51:...| 74|        72|         44|         61|           51|Tweet fictício ge...|
|       54|             71|2024-07-03T19:12:...|  1|        39|         94|         31|           49|Este é um tweet f...|
|       20|             89|2024-07-03T14:58:...|  4|        39|         37|         74|           97|Tweet fictício cr...|
|        8|             36|2024-07-03T06:51:...| 24|        22|          0|         89|           48|Um terceiro tweet...|
+---------+-----

In [22]:
from pyspark.sql import functions as f

In [ ]:
# Novo dataframe para elaborar o relatório

# Etapa 1: agrupar as informações
# groupby unindo pela coluna de data (coluna 'created_at')
# primeiro functions: transforma para um valor de data e cria um apelido/novo nome para a coluna
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))
#....

In [ ]:
# Etapa 2: agregar informações/unir os dados
# agg: como vai agregar os dados e criar novas colunas
# primeira info: quantas pessoas distintas estão fazendo tweets em relação aquele assunto
# para isso usa a coluna de author_id que identifica de maneira única os autores 
# se contar apenas os ids terá apenas o número de tweets, logo precisa ser de uma maneira específica/única - countDistinct
# tb coloca o nome que quer para essa coluna
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"))
#...

In [ ]:
# Outras infos são relacionadas a uma contagem que já existem (likes, quotes, replies, retweets) SÓ QUE POR DATAS
# necessário somar todas as que aconteceram na mesma data - sum

tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet"))
#...

In [23]:
# é relevante saber que dia da semana que aconteceramas ocorrencias
# para isso usa withColumn("weekday") pq quer saber o dia da semana
# f.date_format para formatar uma data (que é o created_date)
# o "E" indica que queremos os dias da semana
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))

In [24]:
tweet_conversas.show(5)

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2024-05-30|      10|   497|    408|    436|      583|    Thu|
|  2024-05-29|       9|   534|    581|    428|      474|    Wed|
|  2024-06-02|      45|  2823|   2780|   3366|     3050|    Sun|
|  2024-07-02|      16|  1186|   1283|    971|     1141|    Tue|
|  2024-05-31|      18|   920|    943|   1152|      875|    Fri|
+------------+--------+------+-------+-------+---------+-------+
only showing top 5 rows



> Próximo passo é transformar esse jupyter em um .py